# Audio Construction

In [1]:
# imports
import os
import numpy as np
import pickle
import wave
import math
from pydub import AudioSegment
from pydub.playback import play
import moviepy.editor as mp

In [2]:
# load quantized impacts
load_dir = os.path.join('data', 'quantized_impacts')
vid_name = 'test'
with open(os.path.join(load_dir, f'{vid_name}_quantized_impacts.pkl'), 'rb') as f:
    quantized_impacts = pickle.load(f)
print(quantized_impacts)

[array([0, 1, 2, 3], dtype=int64), array([4, 5, 6, 7], dtype=int64), array([ 8,  9, 10, 11], dtype=int64), array([12, 13, 14, 15], dtype=int64)]


In [3]:
# function to construct audio
def construct_audio(quantized_impacts, instrument_paths, video_length: float, audio_save_path, audio_load_path=None):
    
    # quantized impacts and instrument paths should both have length n_limbs
    assert len(quantized_impacts) == len(instrument_paths)
        
    # if audio_load_path provided, construct audio on top of original audio (we don't have this yet)
    if audio_load_path:
        base_audio = AudioSegment.from_wav(audio_load_path)
        
        # if audio file provided is shorter than video length, then concatenate .wav file
        frames = base_audio.getnframes()
        rate = base_audio.getframerate()
        temp_audio = base_audio
        while (frames / rate) < video_length:
            base_audio = base_audio + temp_audio
            frames = base_audio.getnframes()
        
        
    # otherwise, construct blank audio file
    else:
        base_audio = AudioSegment.silent(duration=(1000*video_length))
    
    # quantized_impacts is a list of numpy arrays
    # each array corresponds to a limb and contains timestamps in seconds of the impact points
    # each limb has a corresponding instrument sound in instrument_paths
    # instrument sounds should occur in the audio file at every time stamp in the respective array
    for impacts, instrument_path in zip(quantized_impacts, instrument_paths):
        impact_sound = AudioSegment.from_wav(instrument_path)
        
        print(impacts)
        print(instrument_path)
        
        # for each time stamp in array, make sound in file & combine sound file with base_audio
        for time in impacts:
            beat = AudioSegment.silent(duration=(1000*time)) + impact_sound
            base_audio = base_audio.overlay(beat)

            
    final_sound = base_audio
    
    # save audio to audio_save_path
    final_sound.export(audio_save_path, format='wav')
    
    
    success = True
    return success


In [4]:
# stripping sound from video and re-saving
def strip_video(video_load_path, video_save_path, audio_save_path=None):
    
    video = mp.VideoFileClip(video_load_path)
    
    if audio_save_path != None:
        video.audio.write_audiofile(audio_save_path)
    
    video_no_audio = video.without_audio()
    video_no_audio.write_videofile(video_save_path)
    

In [5]:
# combining video and audio
def combine_audio_video(video_load_path, audio_load_path, combined_save_path):
    video = mp.VideoClip(video_load_path)
    audio = mp.AudioFileClip(audio_load_path)
    
    combined = video.set_audio(audio)
    combined.write_videofile(combined_save_path, fps=fps)

In [6]:
# testing

# get instrument paths
instrument_dir = os.path.join('audio', 'instruments')
instrument_names = ['hihat.wav', 'cowbell.wav', 'kick.wav', 'snare.wav']
instrument_paths = [os.path.join(instrument_dir, name) for name in instrument_names]

# create save directory
vid_name = 'test'
audio_save_dir = os.path.join('audio', 'constructed')
if not os.path.exists(audio_save_dir):
    os.makedirs(audio_save_dir)
audio_save_path = os.path.join(audio_save_dir, f'{vid_name}_constructed_audio.wav')

# get load directory (won't use this until we have original audio ready)
audio_load_dir = os.path.join('audio', 'original')
audio_load_path = os.path.join(audio_load_dir, f'{vid_name}_original_audio.wav')

construct_audio(quantized_impacts, instrument_paths, 16, audio_save_path)


# https://github.com/jiaaro/pydub/issues/209
#   getting permission denied error when try to test with play(AudioSegment.from_wav(audio_save_path)); need to look @ changing TEMPDIR
# play(AudioSegment.from_wav(audio_save_path)) """


[0 1 2 3]
audio\instruments\hihat.wav
[4 5 6 7]
audio\instruments\cowbell.wav
[ 8  9 10 11]
audio\instruments\kick.wav
[12 13 14 15]
audio\instruments\snare.wav


True